<a href="https://colab.research.google.com/github/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_NG_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PBMC_NG_2 dataset: Processes the BUG files into files prepared for use in R**

This notebook processes the output from the fastq file processing for this dataset. The data produced here is pre-generated and downloaded by the figure generation code. The purpose of this processing step is to prepare the data for figure generation, by filtering the data and producing downsampled datasets in addition to the original one. 

Steps:
1. Clone the code repo and download data to process
2. Prepare the R environment
3. Process the data
4. Generate statistics for the dataset

**1. Clone the code repo and download data to process**

In [1]:
![ -d "GRNP_2020" ] && rm -r GRNP_2020

!git clone https://github.com/pachterlab/GRNP_2020.git


Cloning into 'GRNP_2020'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (326/326), done.
remote: Compressing objects: 100% (277/277), done.
remote: Total 749 (delta 205), reused 90 (delta 49), pack-reused 423
Receiving objects: 100% (749/749), 7.41 MiB | 5.30 MiB/s, done.
Resolving deltas: 100% (433/433), done.


In [2]:
#download BUG data from Zenodo
!mkdir data
!cd data && wget https://zenodo.org/record/3924675/files/PBMC_NG_2.zip?download=1 && unzip 'PBMC_NG_2.zip?download=1' && rm 'PBMC_NG_2.zip?download=1'

--2020-07-01 23:41:09--  https://zenodo.org/record/3924675/files/PBMC_NG_2.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 477401560 (455M) [application/octet-stream]
Saving to: ‘PBMC_NG_2.zip?download=1’

PBMC_NG_2.zip?downl 100%[===================>] 455.29M  8.31MB/s    in 49s     

2020-07-01 23:42:01 (9.26 MB/s) - ‘PBMC_NG_2.zip?download=1’ saved [477401560/477401560]

Archive:  PBMC_NG_2.zip?download=1
   creating: PBMC_NG_2/
   creating: PBMC_NG_2/bus_output/
  inflating: PBMC_NG_2/bus_output/bug.txt  
  inflating: PBMC_NG_2/bus_output/coll.genes.txt  
  inflating: PBMC_NG_2/bus_output/transcripts_to_genes.txt  


In [3]:
#Check that download worked
!cd data && ls -l && cd PBMC_NG_2/bus_output && ls -l

total 4
drwxr-xr-x 3 root root 4096 Jul  1 00:00 PBMC_NG_2
total 2210072
-rw-r--r-- 1 root root 2254025859 Jul  1 01:12 bug.txt
-rw-r--r-- 1 root root     738211 Jun 30 22:58 coll.genes.txt
-rw-r--r-- 1 root root    8335642 Jun 30 22:44 transcripts_to_genes.txt


**2. Prepare the R environment**

In [4]:
#switch to R mode
%reload_ext rpy2.ipython


/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [5]:
#install the R packages
%%R
sourcePath = "GRNP_2020/NotebookAdaptedRCode/"
install.packages("qdapTools")
install.packages("dplyr")
install.packages("stringdist")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘bitops’, ‘chron’, ‘data.table’, ‘RCurl’, ‘XML’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/bitops_1.0-6.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 8734 bytes

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

**3. Process the data**

Here we discard multimapped UMIs and all UMIs belonging to cells with fewer than 200 UMIs. We also precalculate gene expression, fraction of single-copy molecules etc. and save as stats (statistics). These can later be used when generating figures. We also generate down-sampled BUGs.

In [6]:
#create output directory
!mkdir figureData

In [7]:
#First set some path variables
%%R
source("GRNP_2020/RCode/pathsGoogleColab.R")


In [8]:
#Process and filter the BUG file
%%R
source(paste0(sourcePath, "BUGProcessingHelpers.R"))
createStandardBugsData(paste0(dataPath,"PBMC_NG_2/"), "PBMC_NG_2", c(0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1))



R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘qdapTools’


R[write to console]: The following object is masked from ‘package:dplyr’:

    id




[1] "Generating data for PBMC_NG_2"
[1] "Reading BUG from data/PBMC_NG_2/ ..."
[1] "Filtering multi-mapped reads..."
[1] "Fraction multi-mapped reads: 0.168672376634181"
[1] "Converting genes..."
[1] "Done"
[1] "Down-sampling in total 7 bugs:"
[1] "1: Down-sampling to 0.05"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "2: Down-sampling to 0.1"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



...

[1] "6: Down-sampling to 0.8"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "7: Down-sampling to 1"
[1] "saving BUG..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "Done"
[1] "Saving stats..."


**4. Generate statistics for the dataset**

Here we create a file with various statistics for the dataset, which is used for generating table S2. It also contains some additional information about the dataset. Generation of this file may take several hours.

In [9]:
%%R
source(paste0(sourcePath, "GenBugSummary.R"))
genBugSummary("PBMC_NG_2", "FGF23", "RPS10", 10)

[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 10

In [11]:
!cd figureData/PBMC_NG_2 && ls -l && cat ds_summary.txt

total 1798164
-rw-r--r-- 1 root root 406579016 Jul  2 00:18 Bug_100.RData
-rw-r--r-- 1 root root 125831368 Jul  1 23:57 Bug_10.RData
-rw-r--r-- 1 root root 204713993 Jul  2 00:00 Bug_20.RData
-rw-r--r-- 1 root root 297224849 Jul  2 00:04 Bug_40.RData
-rw-r--r-- 1 root root  70844738 Jul  1 23:55 Bug_5.RData
-rw-r--r-- 1 root root 350274067 Jul  2 00:09 Bug_60.RData
-rw-r--r-- 1 root root 383612943 Jul  2 00:15 Bug_80.RData
-rw-r--r-- 1 root root       980 Jul  2 03:33 ds_summary.txt
-rw-r--r-- 1 root root   2215687 Jul  2 00:18 Stats.RData
Dataset: PBMC_NG_2

totUMIs: 40658153
totCells: 5364
totCounts: 155012999
countsPerUMI: 3.81259323314564
UMIsPerCell: 7579.8197240865
countsPerCell: 28898.769388516
totFracOnes: 0.247950982918481
FracMolWithUMIDistToNeighborH: 73, 707, 1136, 84, 0, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborL: 85, 645, 1153, 111, 6, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborHFrac: 0.0365, 0.3535, 0.568, 0.042, 0, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborLFrac: 0.0425